In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE159000"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE159000"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE159000.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE159000.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE159000.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of non-recurrent human glioblastoma tumorspheres"
!Series_summary	"Samples were obtained from 23 non-recurrent GBM patients treated at Severance Hospital"
!Series_overall_design	"Samples were obtained from 23 non-recurrent GBM patients treated at Severance Hospital"
Sample Characteristics Dictionary:
{0: ['tissue: Brain'], 1: ['Sex: M', 'Sex: F'], 2: ['age: 53', 'age: 51', 'age: 68', 'age: 61', 'age: 49', 'age: 56', 'age: 65', 'age: 11', 'age: 69', 'age: 70', 'age: 57', 'age: 67', 'age: 52', 'age: 42', 'age: 75', 'age: 50', 'age: 48', 'age: 62']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional, List, Dict, Any, Callable

# 1. Determine if gene expression data is available
is_gene_available = True  # Gene expression data is likely available as the title suggests "Gene expression profiles"

# 2. Determine data availability and conversion functions
# 2.1 Data Availability
# We don't have an explicit row that identifies GBM status since all patients have it
trait_row = None  # Trait is not explicitly coded in the characteristics
age_row = 2  # Age information is in row 2
gender_row = 1  # Gender information is in row 1

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary format (1 for having the trait)."""
    # All patients have glioblastoma according to the background information
    return 1

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous format."""
    if not value or ":" not in value:
        return None
    
    try:
        # Extract value after the colon and convert to float
        age_str = value.split(":", 1)[1].strip()
        return float(age_str)
    except (ValueError, IndexError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary format (0 for female, 1 for male)."""
    if not value or ":" not in value:
        return None
    
    # Extract value after the colon
    gender_str = value.split(":", 1)[1].strip().upper()
    
    if gender_str == "F":
        return 0
    elif gender_str == "M":
        return 1
    else:
        return None

# 3. Save metadata
# Since trait_row is None, we rely on background information to determine trait availability
# Background information states all patients have GBM
is_trait_available = True

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# No need for clinical feature extraction since we don't have a proper clinical_data dataframe
# and trait_row is None. We will handle this in later steps with the actual gene expression data.
print(f"Initial filtering completed. Cohort information saved to {json_path}")
print(f"Dataset contains gene expression data: {is_gene_available}")
print(f"Dataset contains trait data: {is_trait_available}")


Initial filtering completed. Cohort information saved to ../../output/preprocess/Glioblastoma/cohort_info.json
Dataset contains gene expression data: True
Dataset contains trait data: True


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 60
Header line: "ID_REF"	"GSM4817125"	"GSM4817126"	"GSM4817127"	"GSM4817128"	"GSM4817129"	"GSM4817130"	"GSM4817131"	"GSM4817132"	"GSM4817133"	"GSM4817134"	"GSM4817135"	"GSM4817136"	"GSM4817137"	"GSM4817138"	"GSM4817139"	"GSM4817140"	"GSM4817141"	"GSM4817142"	"GSM4817143"	"GSM4817144"	"GSM4817145"	"GSM4817146"	"GSM4817147"	"GSM4817148"	"GSM4817149"	"GSM4817150"	"GSM4817151"	"GSM4817152"	"GSM4817153"	"GSM4817154"	"GSM4817155"	"GSM4817156"	"GSM4817157"	"GSM4817158"
First data line: "ILMN_1343291"	13.86646235	13.79568344	13.89969423	14.14217028	13.80018446	13.89187435	13.78265968	13.77749268	13.82485582	14.29516183	14.14268467	14.04071604	13.8376134	13.94562018	13.89574802	13.71926472	13.99971089	13.86952764	13.8631937	14.13215749	14.21943704	14.1160394	14.2703063	14.10571294	14.07107712	15.75998414	15.69032577	13.5802939	13.7254719	14.27520582	14.12386841	14.08193093	14.20840815	14.11377002
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
  

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# The identifiers begin with "ILMN_" which indicates they are Illumina probe IDs
# These are not human gene symbols but microarray probe identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:


Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE159000
Line 6: !Series_title = Gene expression profiles of non-recurrent human glioblastoma tumorspheres
Line 7: !Series_geo_accession = GSE159000
Line 8: !Series_status = Public on Oct 06 2020
Line 9: !Series_submission_date = Oct 05 2020
Line 10: !Series_last_update_date = Jan 06 2023
Line 11: !Series_pubmed_id = 36497392
Line 12: !Series_summary = Samples were obtained from 23 non-recurrent GBM patients treated at Severance Hospital
Line 13: !Series_overall_design = Samples were obtained from 23 non-recurrent GBM patients treated at Severance Hospital
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_contributor = Junseong,,Park
Line 16: !Series_contributor = Seok-Gu,,Kang
Line 17: !Series_sample_id = GSM481712


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# Identifying which columns to use for mapping
# From the annotation preview, we can see:
# - 'ID' column contains probe IDs (ILMN_*) which match the gene expression index
# - 'Symbol' column contains gene symbols

# Step 1: Identify columns for mapping
prob_col = 'ID'
gene_col = 'Symbol'

print(f"Using {prob_col} as probe identifier and {gene_col} as gene symbol for mapping")

# Step 2: Create gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping dataframe with {len(mapping_data)} rows")
print(f"First 5 rows of mapping data:")
print(mapping_data.head(5))

# Step 3: Apply gene mapping to convert probe data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data now contains {len(gene_data)} genes and {gene_data.shape[1]} samples")
print(f"First 5 genes:")
print(gene_data.index[:5])

# Get a sense of the data distribution
print("\nSummary statistics of the first 5 samples:")
print(gene_data.iloc[:, :5].describe())


Using ID as probe identifier and Symbol as gene symbol for mapping
Created mapping dataframe with 44837 rows
First 5 rows of mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene expression data now contains 21461 genes and 34 samples
First 5 genes:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1'], dtype='object', name='Gene')

Summary statistics of the first 5 samples:
         GSM4817125    GSM4817126    GSM4817127    GSM4817128    GSM4817129
count  21461.000000  21461.000000  21461.000000  21461.000000  21461.000000
mean      11.943513     11.731053     11.852193     13.916900     11.669797
std        7.062177      6.932295      7.017661      8.364070      6.894732
min        3.728899      3.678967      3.619433      3.851842      3.406408
25%        7.504871      7.378888  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using data from previous step
# We already have gene_data from the previous step
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Generate clinical data
# Get the SOFT and matrix files again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define conversion functions based on the sample characteristics from step 1
def convert_trait(value):
    # All samples are glioblastoma as per background info
    return 1

def convert_age(value):
    if not value or ':' not in value:
        return None
    # Extract age value after colon
    age_str = value.split(":", 1)[1].strip()
    try:
        # Convert to integer (continuous value)
        return int(age_str)
    except ValueError:
        return None

def convert_gender(value):
    if not value or ':' not in value:
        return None
    # Extract gender value after colon
    gender = value.split(":", 1)[1].strip().upper()
    # Convert to binary: 0 for female, 1 for male
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    else:
        return None
        
# Define row indices based on sample characteristics from step 1
# We found these in the sample characteristics dictionary in step 1
trait_row = 0  # All samples are brain tissue (glioblastoma)
age_row = 2    # Age information is in row 2
gender_row = 1 # Gender information is in row 1

# Extract clinical features
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)

print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (20257, 34)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE159000.csv
Clinical data shape: (3, 34)
Clinical data preview:
              GSM4817125  GSM4817126  GSM4817127  GSM4817128  GSM4817129  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 53.0        51.0        51.0        68.0        61.0   
Gender               1.0         0.0         0.0         1.0         1.0   

              GSM4817130  GSM4817131  GSM4817132  GSM4817133  GSM4817134  ...  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0  ...   
Age                 61.0        49.0        49.0        61.0        56.0  ...   
Gender               1.0         0.0         0.0         1.0         0.0  ...   

              GSM4817149  GSM4817150  GSM4817151  GSM4817152  GSM4817153  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 67.0        52.0        42.0        75.0        61.0   

Data shape after handling missing values: (34, 20260)
Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 57.0
  75%: 64.25
Min: 11.0
Max: 75.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 15 occurrences. This represents 44.12% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data was determined to be unusable or empty and was not saved
